$\newcommand{\ket}[1]{\mathinner{|{#1}\rangle}}$
$\newcommand{\bra}[1]{\mathinner{\langle{#1}|}}$
$\newcommand{\braket}[2]{\mathinner{\langle{#1}\,|\,{#2}\rangle}}$
$\newcommand{\ketbra}[2]{\mathinner{|{#1}\rangle\langle{#2}|}}$

In [ ]:
# 必要なパッケージをインストール
# GPUシミュレータを使う場合は、qisqit-aer を qiskit-aer-gpu に置き換える
%pip install qiskit==1.4.4 qiskit-aer==0.15.1
%pip install pylatexenc
%pip install matplotlib
%pip install qiskit[visualization]

In [ ]:
import numpy as np

from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_aer.noise import pauli_error
from qiskit_aer.noise.errors.base_quantum_error import QuantumChannelInstruction
from qiskit_aer import AerSimulator

from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

n_shots = 10_000 # シミュレーターでのサンプリング回数
if 'GPU' in AerSimulator().available_devices():
    device='GPU'
else:
    device='CPU'
backend_sim = AerSimulator(device=device) # シミュレーターの用意
backend_sim.set_option('method', 'automatic')

エラーチャネルは stane 符号の演習と同じものを使用します。

In [ ]:
def make_bitphase_error_channel(p_error: float, print_flag: bool = True) -> QuantumChannelInstruction:
    bit_flip = pauli_error([('X', p_error), ('I', 1 - p_error)])
    phase_flip = pauli_error([('Z', p_error), ('I', 1 - p_error)])

    bitphase_flip = bit_flip.compose(phase_flip)

    if print_flag:
        print(bitphase_flip)
    return bitphase_flip.to_instruction()

# [9, 1, 3] rotated surface code

図で示すような構造の符号を使って演習を進めます。  
0 〜 8 の番号を記したデータビットと、2 個の測定用アンシラ（9番、10番）を使用して実装します。

（参考）
[こちら](https://github.com/shraggy/qiskit_surface_codes-1/blob/master/tutorials/1_surface_code_encoding.ipynb)のサイトでも同様の実装を行っています。


<img width="500px" src="...
" />

図で示したXスタビライザー $M_6 ,M_4, M_5, M_7$ について測定をすることで
表面符号をエンコードすることができます。

In [ ]:
def surface_code0() -> QuantumCircuit:
    # データビット９個とアンシラ２個を使用
    n_qubits = 9 + 2

    qr = QuantumRegister(n_qubits)
    cr = ClassicalRegister(9)
    # X シンドローム測定結果を保持
    s_x = ClassicalRegister(4)
    circ = QuantumCircuit(qr, cr, s_x)

    # M6
    circ.h(9)
    circ.cx(9, 0)
    circ.cx(9, 1)
    circ.h(9)
    circ.measure(qr[9], s_x[0])
    circ.reset(9)

    # M4
    circ.h(9)
    circ.cx(9, 1)
    circ.cx(9, 2)
    circ.cx(9, 4)
    circ.cx(9, 5)
    circ.h(9)
    circ.measure(qr[9], s_x[1])
    circ.reset(9)

    # M5
    circ.h(9)
    circ.cx(9, 3)
    circ.cx(9, 4)
    circ.cx(9, 6)
    circ.cx(9, 7)
    circ.h(9)
    circ.measure(qr[9], s_x[2])
    circ.reset(9)

    # M7
    circ.h(9)
    circ.cx(9, 7)
    circ.cx(9, 8)
    circ.h(9)
    circ.measure(qr[9], s_x[3])
    circ.reset(9)

    circ.barrier()

    return circ

$\ket{0}$ をエンコードして、測定することで $\ket{0}_L$ の符号語を確認します。

In [ ]:
circ = surface_code0()

circ.measure([0,1,2,3,4,5,6,7,8], [0,1,2,3,4,5,6,7,8])

result = backend_sim.run(transpile(circ.decompose()), shots=n_shots).result()
counts = result.get_counts(0)
plot_histogram(counts)

In [ ]:
pattern = list(counts.items())
for i in range(10):
    print (pattern[i])
print('...')
print('観測パターン数：', len(pattern))

誤りが発生しない状態でシンドローム測定を行ったにもかかわらず、  
X スタビライザーについて固有値 1,-1 が等確率で観測されるために、ヒストグラムが複雑になってしまいます。

これは H ゲートによって X スタビライザーの初期状態が定まっていないためです。  
固有値が -1 の場合、位相が反転している状態でスタビライズされて、以降のシンドローム測定でもエラーがなければ固有値が -1 になります。
（以降の測定でエラーがあれば固有値 0 に変化します）

このように、エラーを検出するためには初期状態と比較することが必要になります。

しかし、これではソフトウェア実装が複雑になるため、  
本演習では位相が反転していることがわかった場合に補正することで、  
エラーがない場合のシンドローム測定結果が常に 0 になるようにします。

少々煩雑な実装ですが、補正の際に `if_test` を使用します。

In [ ]:
def surface_code() -> QuantumCircuit:
    # データビット９個とアンシラ２個を使用
    n_qubits = 9 + 2

    qr = QuantumRegister(n_qubits)
    cr = ClassicalRegister(9)
    # X シンドローム測定結果を保持
    s_x = ClassicalRegister(4)
    circ = QuantumCircuit(qr, cr, s_x)

    # M6
    circ.h(9)
    circ.cx(9, 0)
    circ.cx(9, 1)
    circ.h(9)
    circ.measure(qr[9], s_x[0])
    circ.reset(9)

    # M4
    circ.h(9)
    circ.cx(9, 1)
    circ.cx(9, 2)
    circ.cx(9, 4)
    circ.cx(9, 5)
    circ.h(9)
    circ.measure(qr[9], s_x[1])
    circ.reset(9)

    # M5
    circ.h(9)
    circ.cx(9, 3)
    circ.cx(9, 4)
    circ.cx(9, 6)
    circ.cx(9, 7)
    circ.h(9)
    circ.measure(qr[9], s_x[2])
    circ.reset(9)

    # M7
    circ.h(9)
    circ.cx(9, 7)
    circ.cx(9, 8)
    circ.h(9)
    circ.measure(qr[9], s_x[3])
    circ.reset(9)

    circ.barrier()

    # シンドローム測定の結果に応じて補正する
    with circ.if_test((s_x, 1)):
        circ.z(0)
    with circ.if_test((s_x, 2)):
        circ.z(2)
    with circ.if_test((s_x, 3)):
        circ.z(1)
    with circ.if_test((s_x, 4)):
        circ.z(3)
    with circ.if_test((s_x, 5)):
        circ.z(0)
        circ.z(3)
    with circ.if_test((s_x, 6)):
        circ.z(4)
    with circ.if_test((s_x, 7)):
        circ.z(1)
        circ.z(3)
    with circ.if_test((s_x, 8)):
        circ.z(8)
    with circ.if_test((s_x, 9)):
        circ.z(0)
        circ.z(8)
    with circ.if_test((s_x, 10)):
        circ.z(2)
        circ.z(8)
    with circ.if_test((s_x, 11)):
        circ.z(1)
        circ.z(8)
    with circ.if_test((s_x, 12)):
        circ.z(7)
    with circ.if_test((s_x, 13)):
        circ.z(0)
        circ.z(7)
    with circ.if_test((s_x, 14)):
        circ.z(2)
        circ.z(7)
    with circ.if_test((s_x, 15)):
        circ.z(1)
        circ.z(7)

    circ.barrier()

    # 再測定
    # M6
    circ.h(9)
    circ.cx(9, 0)
    circ.cx(9, 1)
    circ.h(9)
    circ.measure(qr[9], s_x[0])
    circ.reset(9)

    # M4
    circ.h(9)
    circ.cx(9, 1)
    circ.cx(9, 2)
    circ.cx(9, 4)
    circ.cx(9, 5)
    circ.h(9)
    circ.measure(qr[9], s_x[1])
    circ.reset(9)

    # M5
    circ.h(9)
    circ.cx(9, 3)
    circ.cx(9, 4)
    circ.cx(9, 6)
    circ.cx(9, 7)
    circ.h(9)
    circ.measure(qr[9], s_x[2])
    circ.reset(9)

    # M7
    circ.h(9)
    circ.cx(9, 7)
    circ.cx(9, 8)
    circ.h(9)
    circ.measure(qr[9], s_x[3])
    circ.reset(9)

    circ.barrier()

    return circ


if_test の判定において and/or 条件を使えないため、数値で場合分けしています。
例えば、測定結果が 5 の場合、$M_5, M_6$ の固有値が -1 なので、0番と3番（または6番）の位相を反転させます。

上記の補正付きエンコード回路を使って、改めてヒストグラムを作成します。

シンドローム測定結果が 0 になったので、$\ket{0}_L$ の状態が見やすくなります。

In [ ]:
circ = surface_code()
circ.measure([0,1,2,3,4,5,6,7,8], [0,1,2,3,4,5,6,7,8])

result = backend_sim.run(transpile(circ.decompose()), shots=n_shots).result()
counts = result.get_counts(0)
plot_histogram(counts)

# 誤り検出・訂正
ここで 3 番のデータビットに対して、エラーを発生させて測定を行います。

シンドローム測定結果の変化を見るためにエラー発生の前後であわせて 2 回測定を行います。(3 番のビットに隣接しない Z スタビライザー $M_1, M_3$ は省略)
- エラーがない場合は、前後で同じ結果
- ビット反転エラーでは $M_0, M_2$ スタビライザーの固有値が変化
- 位相反転エラーでは $M_5$ スタビライザーの固有値が変化

という結果になるはずです。

In [ ]:
circ = surface_code()

# Z スタビライザーの事前測定
circ.cx(0, 10)
circ.cx(1, 10)
circ.cx(3, 10)
circ.cx(4, 10)
circ.measure(10, 0)
circ.reset(10)

circ.cx(3, 10)
circ.cx(6, 10)
circ.measure(10, 1)
circ.reset(10)


# エラーが発生する部分
bitphase_flip = make_bitphase_error_channel(p_error = 0.3)
for i in [3]:
    circ.append(bitphase_flip, [i])


# Z スタビライザーの事後測定
circ.cx(0, 10)
circ.cx(1, 10)
circ.cx(3, 10)
circ.cx(4, 10)
circ.measure(10, 2)
circ.reset(10)

circ.cx(3, 10)
circ.cx(6, 10)
circ.measure(10, 3)
circ.reset(10)

# X スタビライザーの事後測定
circ.h(9)
circ.cx(9, 0)
circ.cx(9, 1)
circ.h(9)
circ.measure(9, 4)
circ.reset(9)

circ.h(9)
circ.cx(9, 1)
circ.cx(9, 2)
circ.cx(9, 4)
circ.cx(9, 5)
circ.h(9)
circ.measure(9, 5)
circ.reset(9)

circ.h(9)
circ.cx(9, 3)
circ.cx(9, 4)
circ.cx(9, 6)
circ.cx(9, 7)
circ.h(9)
circ.measure(9, 6)
circ.reset(9)

circ.h(9)
circ.cx(9, 7)
circ.cx(9, 8)
circ.h(9)
circ.measure(9, 7)
circ.reset(9)


result = backend_sim.run(transpile(circ.decompose()), shots=n_shots).result()
counts = result.get_counts(0)
plot_histogram(counts)

ヒストグラムにより、
- エラーなし
- 位相エラー・ビットエラーのいずれか発生
- 位相エラー・ビットエラーの両方発生

の３パターンに別れており、確率も想定どおりであることが見えてきます。

# 演習
1. 他の 1 個のデータビットにエラーを発生させてシンドローム測定をします。
1. 2 個のデータビットにエラーを発生させてシンドローム測定をします。エラー訂正はできるでしょうか。

In [ ]:
# 演習
circ = surface_code()

# エラーが発生する部分
bitphase_flip = make_bitphase_error_channel(p_error = 0.3)
for i in [3]:
    circ.append(bitphase_flip, [i])

# Z スタビライザーの事後測定


# X スタビライザーの事後測定



# 測定
result = backend_sim.run(transpile(circ.decompose()), shots=n_shots).result()
counts = result.get_counts(0)
plot_histogram(counts)


# 論理ビット操作

## 論理 X ゲート
符号を形成する正方形で、縦に並んだ 3 個のデータビットに対してビット反転させることで実装できます。
結果、$\ket{1}_L$ が観測されます。

In [ ]:
circ = surface_code()
circ.x(0)
circ.x(3)
circ.x(6)

circ.measure([0,1,2,3,4,5,6,7,8], [0,1,2,3,4,5,6,7,8])

result = backend_sim.run(transpile(circ.decompose()), shots=n_shots).result()
counts = result.get_counts(0)

counts2 = {k[5:]: 0 for k, v in counts.items()}

for k, v in counts.items():
    counts2[k[5:]] += v
plot_histogram(counts2)

## 論理 Z ゲート
符号を形成する正方形で、横に並んだデータビット（例えば 0,1,2）に対して Z ゲートを作用させることで実装できます。  
符号語を見れば、$\ket{0}_L$ についてはどの測定パターンについても、前 3 ビットに 1 が偶数個存在し、
$\ket{1}_L$ については 1 が奇数個存在することから理解できます。

$$
\ket{0}_L = \frac{1}{4} ( \ket{011101110} + \ket{000000011} + \ket{101011000} + \ket{000110101} + \ket{101101110} + \ket{000000000} \\
 + \ket{110000011} + \ket{101101101} + \ket{011011000} + \ket{011101101} + \ket{101011011} + \ket{011011011} + \ket{000110110} \\
 + \ket{110110110} + \ket{110110101} + \ket{110000000} )
$$

$$
\ket{1}_L = \frac{1}{4} ( \ket{111001010} + \ket{001001010} + \ket{001001001} + \ket{111001001} + \ket{010100111} + \ket{100010001} \\
 + \ket{100100111} + \ket{010010010} + \ket{100100100} + \ket{010010001} + \ket{100010010} + \ket{001111111} + \ket{111111100} \\
 + \ket{111111111} + \ket{001111100} + \ket{010100100} )
$$


## 論理 H ゲート
演算としては、各データビットに対して H ゲートを作用させることで実現します。

H ゲートは X 基底と Z 基底を交換する操作になることから、
演算の結果、X スタビライザーが Z スタビライザーに、
Z スタビライザーが X スタビライザーに置き換えられます。

図で示せば、90 度回転させた符号に変わってしまいます。

このままでは実装が複雑になるため、符号を元に戻したいところです。
本演習は（ハードウェア実装を考えずに）ソフトウェア的に、データビットの順序を入れ替えてしまうことで、回転した符号を元に戻します。

具体的には、ビット番号で 0->2, 1->5, 2->8, 3->1, 5->7, 6->0, 7->3, 8->6 に置き換えることで、元の符号に戻します。


In [ ]:
circ = surface_code()

for i in range(9):
    circ.h(i)

circ.barrier()
circ.measure([0,1,2,3,4,5,6,7,8], [0,1,2,3,4,5,6,7,8])

result = backend_sim.run(transpile(circ.decompose()), shots=n_shots).result()
counts = result.get_counts(0)
plot_histogram(counts)


データビットの順序を元に戻して、ヒストグラムを描き直します。
（ソフトウェア的に操作するため量子回路は変更せず、ヒストグラムの描き方だけ変更します）

この結果から、演算後の状態が $\frac{1}{\sqrt{2}} ( \ket{0}_L + \ket{1}_L )$ になっていることが分かります。

In [ ]:
filtered = {k[5:]: v for k, v in counts.items()}
ordered = {k[6]+k[3]+k[0]+k[7]+k[4]+k[1]+k[8]+k[5]+k[2] : v for k, v in filtered.items()}
plot_histogram(ordered)